In [6]:
search_for = 103
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016263723373413086
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 113654321
type: [1, 1, 1, 1, 103] 103
cases of this type: 112550881
10000 20.520768561768367
20000 20.678253410410065
30000 20.422061705773164
40000 76.14566444532686
50000 79.81401964850659
60000 73.74549439716127
70000 72.34646238761958
80000 75.6962550892716
90000 77.70921875347015
100000 74.64993740952745
110000 75.77457950026717
120000 74.90107920893055
130000 73.61054659223498
140000 73.01014467807276
150000 75.05731985553456
160000 75.04597027219393
170000 73.26259395224636
180000 74.62074492198731
190000 70.75448693431355
200000 71.3705102790846
210000 74.57121837512821
220000 73.67477028165581
230000 75.29862277869665
240000 72.9627943258079
250000 71.85907439694797
260000 74.61502214022762
270000 81.92881492050222
280000 80.50498214208366
290000 77.14755165888091
300000 75.73258032794519
310000 73.88765555995249
320000 

3130000 69.02916052406263
3140000 72.45464513376206
3150000 70.04428339670548
3160000 72.62076944134029
3170000 68.98807830375357
3180000 68.08106661385408
3190000 67.4390383717547
3200000 64.28838881771624
3210000 64.41798650677063
3220000 72.85202256374964
3230000 52.06684802450651
3240000 23.93369701686792
3250000 23.93832496765132
3260000 23.83379290591864
3270000 24.274489461349948
3280000 33.98706825251893
3290000 79.58046579741222
3300000 19.370275792214102
3310000 19.446860295409124
3320000 19.53259157760078
3330000 19.52485917121398
3340000 19.344328921797665
3350000 70.55943753781375
3360000 73.02526992575892
3370000 66.33673864432524
3380000 66.23701501969742
3390000 68.98808120942833
3400000 84.34859191369917
3410000 87.77327685761743
3420000 71.64266567114396
3430000 69.17699712204139
3440000 67.98993583786692
3450000 67.6669685519537
3460000 70.79845404848905
3470000 66.3983129059509
3480000 69.80987003325068
3490000 69.75445569841688
3500000 69.3440277880646
3510000 67.0

6300000 68.20636439326965
6310000 71.20606330508406
6320000 71.73428973359776
6330000 68.9631471614202
6340000 65.27704392522182
6350000 64.29829164486105
6360000 67.46352913322609
6370000 63.13131104950247
6380000 61.233207580788914
6390000 61.940723126378025
6400000 64.17796506946723
6410000 63.34494298386449
6420000 66.31241343805112
6430000 69.41506140795603
6440000 69.75953027527073
6450000 64.15548784260328
6460000 71.88080343386534
6470000 78.83667303713673
6480000 29.051131023004167
6490000 23.375174111573106
6500000 23.41199692519528
6510000 23.23758703221084
6520000 23.054325355333287
6530000 54.52094463485577
6540000 81.23914409834823
6550000 72.20682268863817
6560000 67.7093699698083
6570000 69.40486446485718
6580000 66.11309327242894
6590000 69.01144770366814
6600000 73.84470767855109
6610000 19.178123526401674
6620000 19.00289578026768
6630000 19.064329215801372
6640000 18.908130993924754
6650000 18.88825497642821
6660000 70.97933610870588
6670000 73.16263827545407
668000

9450000 61.11036945986573
9460000 62.294541960717154
9470000 67.01193141833505
9480000 65.13967410977291
9490000 63.28229848936859
9500000 66.52311806333246
9510000 63.55875607517942
9520000 61.58581545116965
9530000 65.1848350368023
9540000 67.62370767149852
9550000 60.63426342985473
9560000 60.670637961422166
9570000 64.62972224669024
9580000 60.27874696900444
9590000 60.051687622736964
9600000 64.781631121093
9610000 65.36240311966246
9620000 67.81016812898326
9630000 64.81868245788698
9640000 62.89110542882269
9650000 63.15190862395839
9660000 63.179957498965635
9670000 65.15034735070698
9680000 66.09909189934523
9690000 63.390445533195034
9700000 61.88435416392401
9710000 65.82867917920386
9720000 60.34815412455899
9730000 22.579067697339084
9740000 22.852822964036644
9750000 22.82714266698323
9760000 22.65637464428207
9770000 22.96122021831308
9780000 80.07363877412855
9790000 71.32828055124673
9800000 62.14078195073372
9810000 60.026041721332135
9820000 64.91459129458414
9830000

12500000 55.10445563859046
12510000 54.07101980660439
12520000 56.24888601485761
12530000 64.10051910586746
12540000 69.85021872508085
12550000 8.902181348527167
12560000 8.953830150734861
12570000 8.873922660676659
12580000 8.89537626638733
12590000 8.645502171060741
12600000 55.5790632697164
12610000 68.39968298008598
12620000 59.25776165426351
12630000 58.161632325561015
12640000 56.40453455520043
12650000 56.57441650509876
12660000 55.491299587638956
12670000 55.294053520268164
12680000 57.831185181552954
12690000 59.03565138932768
12700000 60.45110344509163
12710000 56.865779415697
12720000 57.23688898412491
12730000 57.68755251508826
12740000 60.05319068465995
12750000 56.796483527178395
12760000 56.247632258325794
12770000 62.30639222277819
12780000 63.81937421656185
12790000 61.978435167759564
12800000 62.860311073114836
12810000 66.27766795403983
12820000 63.65693313296959
12830000 64.09518612018658
12840000 62.59805174416446
12850000 61.809243648257784
12860000 59.22869816487

...

99430000 9.43586111732914
99440000 9.067204210179211
99450000 9.399178652280582
99460000 9.940901994403363
99470000 9.298546235389253
99480000 9.2014867202556
99490000 8.92770931346847
99500000 8.983896777408937
99510000 8.591552465358342
99520000 8.566503561376605
99530000 9.568311184343756
99540000 5.66621507882558
99550000 3.077641700062891
99560000 3.146654496849537
99570000 3.0483217609378697
99580000 3.069428461736944
99590000 5.606269657895486
99600000 10.392421930386757
99610000 9.017022380016538
99620000 8.418749543819011
99630000 8.277666396010783
99640000 8.659503900806957
99650000 9.15997865100809
99660000 9.093541333281033
99670000 9.130068487345225
99680000 8.523053198319653
99690000 8.492709301040568
99700000 8.212593337600424
99710000 8.509213574128985
99720000 8.492401349716657
99730000 8.857770836716606
99740000 9.092355155965109
99750000 8.703911389160773
99760000 8.887514028453046
99770000 9.578956837170342
99780000 8.60124016700358
99790000 8.120903221582513
998000

102370000 6.556717484180094
102380000 7.351683071228683
102390000 7.603624273689389
102400000 6.990571367314637
102410000 7.447254214695275
102420000 6.496771815176355
102430000 6.1800259414717225
102440000 6.432772939768831
102450000 6.675518673569462
102460000 6.51478252646158
102470000 6.669630511901339
102480000 6.782260920073145
102490000 6.603806233695096
102500000 6.850564245843708
102510000 6.9434475105009215
102520000 6.91886344128906
102530000 6.79836053217119
102540000 6.68298580376802
102550000 6.675917245938467
102560000 6.426165212396144
102570000 6.494488759658992
102580000 7.28887134827553
102590000 7.5438623950078485
102600000 1.7845831710922917
102610000 1.7457636933936345
102620000 1.879832673669994
102630000 1.8965764964286285
102640000 1.934862076634811
102650000 6.7996932641878916
102660000 7.136795506143445
102670000 6.77892695187017
102680000 6.314575841966629
102690000 6.5876357519546085
102700000 7.306376082646297
102710000 7.196409684924623
102720000 6.869960

105290000 4.856147814955115
105300000 4.724038500401683
105310000 5.223871438493351
105320000 4.977330322750927
105330000 5.111419955517047
105340000 4.918784247608111
105350000 4.69176056892856
105360000 4.582684281502406
105370000 4.861494292847289
105380000 4.564264181120694
105390000 4.574413190617952
105400000 4.363915637101034
105410000 4.514918307224651
105420000 5.342794113371127
105430000 1.2774232864897384
105440000 0.728033644353628
105450000 0.7155091001392603
105460000 0.6658327995037635
105470000 0.6943026669255495
105480000 3.5840055876932078
105490000 5.538840982785444
105500000 4.498033031620184
105510000 4.060958177699129
105520000 4.423967253357357
105530000 4.642221792427619
105540000 4.837398579455574
105550000 4.809286044548551
105560000 5.002210268770993
105570000 4.668606010102106
105580000 4.959086968772703
105590000 5.002204331064781
105600000 4.590007083181262
105610000 4.7421374856601295
105620000 4.650275552187443
105630000 4.6087216239368844
105640000 4.54

108190000 4.084052798744268
108200000 1.201964968915065
108210000 1.1907697687297265
108220000 1.1057765294498603
108230000 1.1203955975144706
108240000 1.112016819832583
108250000 2.815029479033907
108260000 3.789792511325955
108270000 3.310744809946954
108280000 3.0504026962261133
108290000 3.135361541530411
108300000 3.1450834007986916
108310000 3.0988229387354385
108320000 3.0915942742820977
108330000 3.1050163482464086
108340000 3.1767515655081073
108350000 3.2194348581121566
108360000 3.0662614479943313
108370000 3.134563333981441
108380000 3.0634576922239463
108390000 3.0899892137365605
108400000 3.182626768612246
108410000 3.112864255958776
108420000 3.0374556075350707
108430000 3.29204934420473
108440000 3.2337561851831675
108450000 3.222082407025112
108460000 3.190462466614425
108470000 3.110104007735888
108480000 3.283334867366625
108490000 3.050610538733681
108500000 3.0265245745997627
108510000 2.9660043016871214
108520000 2.94218369289734
108530000 2.986868841317952
10854

111060000 1.6716644615167446
111070000 1.6205960463992755
111080000 1.6304750901578666
111090000 1.5796691944419012
111100000 1.4856275454323558
111110000 1.6214051551798978
111120000 1.6157428666974638
111130000 1.57634169236709
111140000 1.605463683388889
111150000 1.6733566823886235
111160000 1.5193619143239525
111170000 1.7082816658015054
111180000 1.60378936275459
111190000 1.8457206281150316
111200000 1.7701835991182924
111210000 1.5779159117459456
111220000 1.5895639659067191
111230000 1.5641627114227414
111240000 1.5852412621755136
111250000 1.5770007079224853
111260000 1.5401865823957324
111270000 1.5463660236468448
111280000 1.45892911387574
111290000 1.43767491589731
111300000 1.5500399301255015
111310000 1.571083239211599
111320000 1.6354574521733523
111330000 1.5280996589315599
111340000 1.5917868496568865
111350000 1.57316632166032
111360000 1.5013424832149675
111370000 1.4725595956850317
111380000 1.449545469815274
111390000 1.32071217442335
111400000 1.483030989282396
1